In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import chardet

In [2]:
df = pd.read_csv('../data/denver_listing_dates_with_ct.csv')

df["list_date"] = pd.to_datetime(df["list_date"])
df["sale_date"] = pd.to_datetime(df["sale_date"])

In [3]:
prop = pd.read_csv('../data/denver_property_info.csv')
prop = prop.rename(columns={"cc_property_id": "property_id"})

/Users/namluu/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
prop.head()

,property_id,data_source,address,city,zipcode,cc_property_address_county,fips,apn,property_guid,latitude,...,below_grade_bed_count,below_grade_sqft,current_use,fencing_description,frontage_length,property_condition,property_quality,senior_community_indicator,zoning_code,zoning_description
0,138572150,dataharmony,4950 Dover St,Arvada,80002,JEFFERSON,8059.0,004274,NaN,39.78644,...,NaN,NaN,Multi-Family,Partial,NaN,New Construction,Stucco,NaN,R-2,NaN
1,322446242,dataharmony,5260 Estes Cir,Arvada,80002,NaN,8059.0,NaN,NaN,39.79225,...,NaN,NaN,NaN,NaN,NaN,NaN,"Frame,Vinyl Siding",NaN,R,NaN
2,138560546,dataharmony,5622 Ammons St,Arvada,80002,NaN,8059.0,NaN,NaN,39.79995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,138555680,dataharmony,5615 Jellison St,Arvada,80002,NaN,8059.0,NaN,NaN,39.79878,...,3.0,1120.0,NaN,Full,NaN,Updated/Remodeled,Brick,NaN,RES,NaN
4,138576011,dataharmony,5303 Arbutus St,Arvada,80002,NaN,8059.0,NaN,NaN,39.79271,...,NaN,1040.0,NaN,NaN,NaN,NaN,Brick,NaN,RES,R-1A


In [5]:
df.head()

,property_id,zipcode,latitude,longitude,list_date,sale_date,withdrawn_date,expired_date,cancelled_date,current_status,sale_price,geometry,index_right,ct_key
0,666680976,80113,39.65232,-104.97613,2016-03-01,NaT,NaN,2016-03-01,NaN,Expired,NaN,POINT (-104.97613 39.65232),10693.0,8.005006e+09
1,138826052,80123,39.62523,-105.08648,2016-03-01,NaT,NaN,2016-03-01,NaN,Expired,NaN,POINT (-105.08648 39.62523),9869.0,8.059012e+09
2,666683709,80204,39.74122,-105.05241,2016-03-01,NaT,NaN,2016-03-01,NaN,Expired,NaN,POINT (-105.05241 39.74122),10476.0,8.031001e+09
3,137981678,80241,39.91465,-104.93076,2016-03-01,2016-04-21,NaN,NaN,NaN,Sale,320000.0,POINT (-104.93076 39.91465),9719.0,8.001009e+09
4,410875086,80202,39.74783,-104.98640,2016-03-01,2016-05-02,NaN,NaN,NaN,Sale,475000.0,POINT (-104.9864 39.74783),10465.0,8.031002e+09


In [7]:
houses = pd.merge(prop, df, on='property_id')

In [8]:
percent_notnull = houses.notnull().sum() * 100 / len(df)
notnull_df = pd.DataFrame({'column_name': houses.columns, 'percent_notnull': percent_notnull})

In [10]:
notnull_df[notnull_df['percent_notnull'] >= 50]

,column_name,percent_notnull
property_id,property_id,97.137181
data_source,data_source,97.137181
address,address,97.136953
city,city,97.135588
zipcode_x,zipcode_x,97.137181
fips,fips,96.325442
latitude_x,latitude_x,97.072779
longitude_x,longitude_x,97.072779
sqft,sqft,91.628189
state,state,97.081882


In [11]:
truncated_df = houses[['property_id', 'zipcode_x', 'fips', 'latitude_x', 'longitude_x', 'sqft', 'property_type', 'has_central_air', 'has_jacuzzi', 'has_pool', 'has_solar', 'has_garage', 'current_status', 'list_date', 'sale_date']]


In [12]:
truncated_df

,property_id,zipcode_x,fips,latitude_x,longitude_x,sqft,property_type,has_central_air,has_jacuzzi,has_pool,has_solar,has_garage,list_date,current_status,list_date,sale_date
0,138572150,80002,8059.0,39.78644,-105.09357,NaN,OTHER,False,False,False,False,False,2017-09-01,Expired,2017-09-01,NaT
1,138572150,80002,8059.0,39.78644,-105.09357,NaN,OTHER,False,False,False,False,False,2020-09-28,Active,2020-09-28,NaT
2,322446242,80002,8059.0,39.79225,-105.09622,NaN,CONDO,False,False,False,False,False,2017-01-15,Sale,2017-01-15,2017-02-16
3,322446242,80002,8059.0,39.79225,-105.09622,NaN,CONDO,False,False,False,False,False,2019-07-31,Sale,2019-07-31,2019-09-24
4,138560546,80002,8059.0,39.79995,-105.08691,1687.0,SINGLE,False,False,False,False,False,2018-05-17,Sale,2018-05-17,2018-07-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426842,503644833,80643,8123.0,40.10629,-104.52506,1288.0,SINGLE,False,False,False,False,False,2018-07-31,Sale,2018-07-31,2018-09-14
426843,551276799,80643,8123.0,40.03606,-104.47066,1810.0,SINGLE,False,False,False,False,False,2017-08-17,Sale,2017-08-17,2017-10-11
426844,139419740,80643,8123.0,40.10923,-104.51945,1000.0,SINGLE,False,False,False,False,False,2018-05-11,Sale,2018-05-11,2018-07-30
426845,480458508,80643,8123.0,40.10286,-104.52007,1620.0,SINGLE,False,False,False,False,False,2017-02-18,Sale,2017-02-18,2017-03-24


In [13]:
truncated_df = truncated_df.rename(columns={"latitude_x": "latitude", 'longitude_x': 'longitude', 'zipcode_x': 'zipcode'})

In [14]:
clean_df = truncated_df.dropna()

In [15]:
clean_df

,property_id,zipcode,fips,latitude,longitude,sqft,property_type,has_central_air,has_jacuzzi,has_pool,has_solar,has_garage,list_date,current_status,list_date,sale_date
4,138560546,80002,8059.0,39.79995,-105.08691,1687.0,SINGLE,False,False,False,False,False,2018-05-17,Sale,2018-05-17,2018-07-09
6,138555680,80002,8059.0,39.79878,-105.10757,2220.0,SINGLE,False,False,False,False,False,2019-12-27,Sale,2019-12-27,2020-01-31
7,138576011,80002,8059.0,39.79271,-105.14868,1200.0,SINGLE,False,False,False,False,False,2019-03-30,Sale,2019-03-30,2019-05-17
8,138574571,80002,8059.0,39.79170,-105.13083,2127.0,SINGLE,False,False,False,False,False,2017-09-28,Sale,2017-09-28,2017-10-31
9,138574571,80002,8059.0,39.79170,-105.13083,2127.0,SINGLE,False,False,False,False,False,2019-09-11,Sale,2019-09-11,2019-10-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426840,577709232,80643,8001.0,39.95822,-104.54549,1996.0,SINGLE,False,False,False,False,False,2018-05-07,Sale,2018-05-07,2018-07-20
426842,503644833,80643,8123.0,40.10629,-104.52506,1288.0,SINGLE,False,False,False,False,False,2018-07-31,Sale,2018-07-31,2018-09-14
426843,551276799,80643,8123.0,40.03606,-104.47066,1810.0,SINGLE,False,False,False,False,False,2017-08-17,Sale,2017-08-17,2017-10-11
426844,139419740,80643,8123.0,40.10923,-104.51945,1000.0,SINGLE,False,False,False,False,False,2018-05-11,Sale,2018-05-11,2018-07-30


In [18]:
clean_df.to_csv('../data/denver_prop_listing_clean.csv', index=False)